## Notebook for making predictions with deleted constant y samples


In [49]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import numpy as np
from sklearn.ensemble import StackingRegressor
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import BaggingRegressor


### Reading dataset stored in other notebooks

In [50]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [51]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'dew_or_rime:idx', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p', 'elevation:m', 'fresh_snow_12h:cm',
       'fresh_snow_1h:cm', 'fresh_snow_24h:cm', 'fresh_snow_3h:cm',
       'fresh_snow_6h:cm', 'is_day:idx', 'is_in_shadow:idx',
       'msl_pressure:hPa', 'precip_5min:mm', 'precip_type_5min:idx',
       'pressure_100m:hPa', 'pressure_50m:hPa', 'prob_rime:p',
       'rain_water:kgm2', 'relative_humidity_1000hPa:p', 'sfc_pressure:hPa',
       'snow_depth:cm', 'snow_drift:idx', 'snow_melt_10min:mm',
       'snow_water:kgm2', 'sun_azimuth:d', 'sun_elevation:d',
       'super_cooled_liquid_water:kgm2', 't_1000hPa:K', 'total_cloud_cover:p',
       'visibility:m', 'wind_speed_10m:ms', 'wind_speed_u_10m:ms',
       'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:m

In [52]:
# remove special characters from column names
X_A_train.columns = X_A_train.columns.str.replace(':', '_')
X_B_train.columns = X_B_train.columns.str.replace(':', '_')
X_C_train.columns = X_C_train.columns.str.replace(':', '_')

X_A_test.columns = X_A_test.columns.str.replace(':', '_')
X_B_test.columns = X_B_test.columns.str.replace(':', '_')
X_C_test.columns = X_C_test.columns.str.replace(':', '_')

## Catboost models
will try with two models:
-   One for dataset A
-   One for B and C

In [53]:
catModel_a = CatBoostRegressor()
catModel_b = CatBoostRegressor()
catModel_c = CatBoostRegressor()

In [54]:
# Base models
catModel_a0 = CatBoostRegressor()
catModel_b0 = CatBoostRegressor()
catModel_c0 = CatBoostRegressor()

In [55]:
rfr_a = RandomForestRegressor()
rfr_b = RandomForestRegressor()
rfr_c = RandomForestRegressor()

In [56]:
xgb_a = XGBRegressor()
xgb_b = XGBRegressor()
xgb_c = XGBRegressor()

In [57]:
nn_a = MLPRegressor()
nn_b = MLPRegressor()
nn_c = MLPRegressor()

In [58]:
gbm_a = lgb.LGBMRegressor()
gbm_b = lgb.LGBMRegressor()
gbm_c = lgb.LGBMRegressor()

In [59]:
# meta models
metaModel_a = LinearRegression()
metaModel_b = LinearRegression()
metaModel_c = LinearRegression()

In [60]:
# stacking classifier
stacking_model_a = StackingRegressor(
    estimators=[('rfr', rfr_a), ('nn', nn_a), ('gbm', gbm_a)],
    final_estimator=metaModel_a
)

stacking_model_b = StackingRegressor(
    estimators=[('rfr', rfr_b), ('nn', nn_b), ('gbm', gbm_b)],
    final_estimator=metaModel_b
)

stacking_model_c = StackingRegressor(
    estimators=[('rfr', rfr_c), ('nn', nn_c), ('gbm', gbm_c)],
    final_estimator=metaModel_c
)

In [61]:
# fit stacking models
stacking_model_a.fit(X_A_train, Y_A_train)
stacking_model_b.fit(X_B_train, Y_B_train)
stacking_model_c.fit(X_C_train, Y_C_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004377 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7221
[LightGBM] [Info] Number of data points in the train set: 23224, number of used features: 44
[LightGBM] [Info] Start training from score 925.459494
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7153
[LightGBM] [Info] Number of data points in the train set: 18579, number of used features: 44
[LightGBM] [Info] Start training from score 948.228123
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002562 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7190
[LightGBM] [Info] Number of data points in the train set: 18579, number of used features: 44
[LightGBM] [Info] Start

StackingRegressor(estimators=[('rfr', RandomForestRegressor()),
                              ('nn', MLPRegressor()),
                              ('gbm', LGBMRegressor())],
                  final_estimator=LinearRegression())

In [62]:
# fit base models
catModel_a0.fit(X_A_train, Y_A_train)
catModel_b0.fit(X_B_train, Y_B_train)
catModel_c0.fit(X_C_train, Y_C_train)

Learning rate set to 0.067297
0:	learn: 1247.6744460	total: 15.5ms	remaining: 15.4s
1:	learn: 1186.0260545	total: 19.7ms	remaining: 9.81s
2:	learn: 1130.6434933	total: 23.8ms	remaining: 7.92s


3:	learn: 1078.1808484	total: 28ms	remaining: 6.97s
4:	learn: 1029.7823566	total: 32.4ms	remaining: 6.45s
5:	learn: 985.1533261	total: 36.9ms	remaining: 6.12s
6:	learn: 943.6513425	total: 41.4ms	remaining: 5.88s
7:	learn: 906.2632481	total: 45.9ms	remaining: 5.69s
8:	learn: 872.3898611	total: 51.1ms	remaining: 5.63s
9:	learn: 840.7465563	total: 56.6ms	remaining: 5.6s
10:	learn: 812.1274775	total: 61.4ms	remaining: 5.52s
11:	learn: 784.7392676	total: 65.9ms	remaining: 5.42s
12:	learn: 760.7637750	total: 70.7ms	remaining: 5.37s
13:	learn: 738.6033910	total: 75.8ms	remaining: 5.33s
14:	learn: 718.7709395	total: 79.6ms	remaining: 5.23s
15:	learn: 700.9002452	total: 83.7ms	remaining: 5.15s
16:	learn: 684.6465070	total: 87.5ms	remaining: 5.06s
17:	learn: 669.7781852	total: 91.7ms	remaining: 5s
18:	learn: 655.3989838	total: 95.9ms	remaining: 4.95s
19:	learn: 643.4918877	total: 99.6ms	remaining: 4.88s
20:	learn: 632.6491832	total: 104ms	remaining: 4.84s
21:	learn: 622.5557189	total: 107ms	rema

## Preds

In [63]:
print(X_A_test.columns)

Index(['absolute_humidity_2m_gm3', 'air_density_2m_kgm3',
       'clear_sky_energy_1h_J', 'clear_sky_rad_W', 'dew_or_rime_idx',
       'dew_point_2m_K', 'diffuse_rad_W', 'diffuse_rad_1h_J', 'direct_rad_W',
       'direct_rad_1h_J', 'effective_cloud_cover_p', 'elevation_m',
       'fresh_snow_12h_cm', 'fresh_snow_1h_cm', 'fresh_snow_24h_cm',
       'fresh_snow_3h_cm', 'fresh_snow_6h_cm', 'is_day_idx',
       'is_in_shadow_idx', 'msl_pressure_hPa', 'precip_5min_mm',
       'precip_type_5min_idx', 'pressure_100m_hPa', 'pressure_50m_hPa',
       'prob_rime_p', 'rain_water_kgm2', 'relative_humidity_1000hPa_p',
       'sfc_pressure_hPa', 'snow_depth_cm', 'snow_drift_idx',
       'snow_melt_10min_mm', 'snow_water_kgm2', 'sun_azimuth_d',
       'sun_elevation_d', 'super_cooled_liquid_water_kgm2', 't_1000hPa_K',
       'total_cloud_cover_p', 'visibility_m', 'wind_speed_10m_ms',
       'wind_speed_u_10m_ms', 'wind_speed_v_10m_ms', 'wind_speed_w_1000hPa_ms',
       'direct_rad_W_lag_-23', 'month'

In [64]:
# preds for stacking models
stacking_preds_a = stacking_model_a.predict(X_A_test)
stacking_preds_b = stacking_model_b.predict(X_B_test)
stacking_preds_c = stacking_model_c.predict(X_C_test)

In [65]:
# preds for base models
pred_a = catModel_a0.predict(X_A_test)
pred_b = catModel_b0.predict(X_B_test)
pred_c = catModel_c0.predict(X_C_test)

In [66]:
pred_A_new = np.mean([pred_a, stacking_preds_a], axis=0)
pred_B_new = np.mean([pred_b, stacking_preds_b], axis=0)
pred_C_new = np.mean([pred_c, stacking_preds_c], axis=0)

In [67]:
df_A = pd.DataFrame()

df_A["predict"] = pred_A_new
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["predict"] = pred_B_new
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["predict"] = pred_C_new
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)

df = df.drop("location", axis=1)

df[df<0] = 0

# NAME THE FILE 
df.to_csv("./ext_preds/stack_4.csv")

df.shape

(2160, 1)